<a href="https://colab.research.google.com/github/oagbohouto/Suivi-data-camp-Elhajam-Agbohouto-Yokam-Koua/blob/main/Analyse_exploratoire_visualisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ASHRAE Energy Predictions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Problématique et but du projet
De nos jours, de plus en plus d'investissements sont réalisés dans le domaine de l'immobilier dans le but de réduire les consommations d'énergie des bâtiments et d'améliorer l'impact environnemental.

Ainsi, les propriétaires d'immeubles peuvent bénéficier de financements basés sur la différence entre la consommation d'énergie réelle du bâtiment et celle qu'il aurait utilisée sans aucuns travaux d'aménagement. Toutefois, les données sur la consommation d'énergie des bâtiments au cas où il n'aurait pas de rénovation ne sont pas disponibles.

Pour résoudre ce problème, des modèles contrefactuels sont développés afin de modéliser la consommation d'énergie d'un bâtiment sans travaux de rénovation.

Le but de ce TP est de construire ces modèles contrefactuels pour les quatre types d'énergie que sont la consommation d'eau froide, d'électricité, d'eau chaude et de vapeur en se basant sur les taux d'utilisation historiques et les conditions météorologiques observées. Il s'agira concrètement de prédire les valeurs de la variable meter_reading pour 1449 bâtiments.

# Chapitre 1: Analyse exploratoire des données

## I- Importation des données

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.stats
import gc
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
    

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Kaggle/train.csv')
weather_train = pd.read_csv('/content/drive/MyDrive/Kaggle/weather_train.csv')
building_metadata = pd.read_csv('/content/drive/MyDrive/Kaggle/building_metadata.csv')

## II- Description des données
L'ensemble de données comprend trois années de relevés de compteurs horaires de plus de mille bâtiments sur plusieurs sites différents à travers le monde. Les données sont regroupées en deux types: les données météorologiques et les données individuelles. Les données météorologiques sont communes à tous les bâtiments d'un même site et les données individuelles sont propres à chacun des bâtiments.

Les données météorologiques sont contenues dans la base weather_train/test.csv et concernent les données de températures, de niveau de précipitation, de vitesse et de direction du vent.

Les données individuelles sont contenues dans les bases building_meta.csv et concernent les données sur l'utilisation primaire du bâtiment, la surface brute du bâtiment, le nombre d'étages des immeubles ou encore l'année d'ouverture du bâtiment.

Les données train.csv décrivent quant à elles contiennent les données sur les quatre différents types de compteur (eau chaude et froide, gaz et électricité) et les consommations de ces énergies de chaque bâtiment.

## III- Etude de chaque table de données

### A- Analyse de la base de données train

Cette base de données contient les 4 variables sivantes:

- building_id: identifie chaque batiment
- meter: le type de compteur. Il y en a 4, electricity, chilled water, hot water et steam
- timestamp: les dates et heures de lecture de compteur; toutes les heures du 01/01/2016 à 00:00:00 au 31/12/2016 à 23:00:00
- meter_reading: les consommations d'énergie rélévées 

In [ ]:
train.info()

Pour avoir une idée de la distribution de la variable d'intérêt qui est la consommation d'énergie (meter_reading), nous faisons un histogramme

In [ ]:
plt.hist(train['meter_reading'])

Nous constatons qu'il y a plein de zéros pour cette variable. Toute fois, l'histogramme ne nous permet pas de bien voir la distribution de cette variable pour les valeurs différentes de zéro. Nous allons étudier les données pour des meter reading égale à zéro séparément des autres dans un premier temps.

In [ ]:
train['meter_reading'].describe()

En regardant la descrirption de cette variable, la moyenne est loin de zéro. On s'intéresse donc maintenant au pourcentage de valeurs qui sont égales à zéro.

In [ ]:
train[train['meter_reading']==0].shape[0] / train.shape[0]

Environ 9.3% des valeurs des compteurs sont égales à zéro. Nous ne pouvons pas dire exactement à quoi attribuer ce phénomène. Il se pourrait que certains de ces valeurs sont égaleas à zéro parcequ'il n'y avait pas de consommation d'énergie, ou par exemple pour le dû au fait qu'en hiver on n'a pas besoin d'eau glacée.

In [ ]:
len(train['meter'])

In [ ]:
train['meter'].replace({0:"electricity",1:"chilledwater",2:"steam",3:"hotwater"},inplace=True)

In [ ]:
meter_dict = {}
for i in train['meter'].unique():
    percent = round(train[train['meter_reading']== 0]['meter'].value_counts()[i] / train['meter'].value_counts()[i],2)
    meter_dict[i] = percent
zero_meter = pd.Series(meter_dict)
sns.barplot(x=zero_meter.index, y= zero_meter)
plt.title("Meters percentage having zero readings")
plt.show()

D'après le graphique ci-dessus, on constate que le compteur d'eau chaude a plus de zéros que les autres compteurs. Mais nous avons à faire au données temporelles, et donc ce n'est peut être pas la meilleur façon de regarder cette variable. Dans ce qui suit, nous allons faire des études plus détaillées.

Nous convertissons tout d'abord la variable timestamp en objet de date et temps, et ensuite attribuons le mois correspondant à chaque date pour pouvoir étudier les tendences mensuelles de meter_reading.

In [ ]:
train['meter'].unique()

train['timestamp2'] = pd.to_datetime(train["timestamp"])
train['month'] = train.timestamp2.dt.month

On représente ensuite séparément pour chaque compteur les valeurs de meter_reading par mois pour voir à quel moment on a ces valeurs sur l'année.

In [ ]:
fig, axs = plt.subplots(2,2, sharey=True, tight_layout=True,figsize=(10,6))

axs[0][0].hist(x ="month",data =train[(train.meter_reading == 0) & (train.meter=="electricity")],bins =12,color = "navajowhite")
axs[0][0].set_title("For electricity")

axs[0][1].hist(x ="month",data =train[(train.meter_reading == 0) & (train.meter=="chilledwater")],bins =12,color = "skyblue")
axs[0][1].set_title("For chilled water")

axs[1][0].hist(x ="month",data =train[(train.meter_reading == 0) & (train.meter=="steam")],bins =12,color = "slategrey")
axs[1][0].set_title("For steam")

axs[1][1].hist(x ="month",data =train[(train.meter_reading == 0) & (train.meter=="hotwater")],bins =12,color = "lightcoral")
axs[1][1].set_title("For hot water")

Comme on peut le voir sur les graphes, et comme attendu, les consommation de 0 change d'un mois à l'autre sur l'année en fonction du type de compteur aussi. Pour le compteur d'électricité, on a des consommation de 0 plutôt au début de l'année jusqu'en mai environ. A partir de juin les consommation de 0 baisse. Les consommations à 0 de "steam" et d'eau chaude ont à peu près la même tendance sur l'année. Les consommations à 0 d'eau glacée sont saisonnières aussi.

En ne séparant pas les consommations à 0 du reste des consommations comme dans les graphiques ci-dessous, on n'arrive pas à voir l'effet des consommations à 0.

In [ ]:
fig, axs = plt.subplots(2,2, sharey=True, tight_layout=True,figsize=(10,6))

axs[0][0].hist(x ="month",data =train[(train.meter=="electricity")],bins =12,color = "navajowhite")
axs[0][0].set_title("For electricity")

axs[0][1].hist(x ="month",data =train[(train.meter=="chilledwater")],bins =12,color = "skyblue")
axs[0][1].set_title("For chilled water")

axs[1][0].hist(x ="month",data =train[(train.meter=="steam")],bins =12,color = "slategrey")
axs[1][0].set_title("For steam")

axs[1][1].hist(x ="month",data =train[(train.meter=="hotwater")],bins =12,color = "lightcoral")
axs[1][1].set_title("For hot water")

Les consommations différentes de 0 ont une tendance plutôt uniforme, comme ci-dessous.

In [ ]:
fig, axs = plt.subplots(2,2, sharey=True, tight_layout=True,figsize=(10,6))

axs[0][0].hist(x ="month",data =train[(train.meter_reading != 0) & (train.meter=="electricity")],bins =12,color = "navajowhite")
axs[0][0].set_title("For electricity")

axs[0][1].hist(x ="month",data =train[(train.meter_reading != 0) & (train.meter=="chilledwater")],bins =12,color = "skyblue")
axs[0][1].set_title("For chilled water")

axs[1][0].hist(x ="month",data =train[(train.meter_reading != 0) & (train.meter=="steam")],bins =12,color = "slategrey")
axs[1][0].set_title("For steam")

axs[1][1].hist(x ="month",data =train[(train.meter_reading != 0) & (train.meter=="hotwater")],bins =12,color = "lightcoral")
axs[1][1].set_title("For hot water")

In [ ]:
sns.kdeplot(train.loc[(train['meter']=='electricity'), 
            'meter_reading'], color='yellow', shade=False, Label='electricity')

sns.kdeplot(train.loc[(train['meter']=='chilledwater'), 
            'meter_reading'], color='b', shade=False, Label='chilledwater')

sns.kdeplot(train.loc[(train['meter']=='steam'), 
            'meter_reading'], color='gray', shade=False, Label='steam')

sns.kdeplot(train.loc[(train['meter']=='hotwater'), 
            'meter_reading'], color='r', shade=False, Label='hotwater')

plt.xlabel('meter_reading') 
plt.ylabel('Probability Density') 

Nous faisons une log transformation sur la variable meter_reading pour avoir des données plus normalisées et une bonne représentation de la densité.

In [ ]:
train['meter_reading_log'] = np.log1p(train['meter_reading'])

In [ ]:
sns.kdeplot(train.loc[(train['meter']=='electricity'), 
            "meter_reading_log"], color='yellow', shade=False, Label='electricity')

sns.kdeplot(train.loc[(train['meter']=='chilledwater'), 
            "meter_reading_log"], color='b', shade=False, Label='chilledwater')

sns.kdeplot(train.loc[(train['meter']=='steam'), 
            "meter_reading_log"], color='gray', shade=False, Label='steam')

sns.kdeplot(train.loc[(train['meter']=='hotwater'), 
            "meter_reading_log"], color='r', shade=False, Label='hotwater')

plt.xlabel('meter_reading_log') 
plt.ylabel('Probability Density') 

In [ ]:
len(set(train['building_id']))

### B- Analyse des données building

Les données building_meta.csv sont composé de 6 variables définies par :

- site_id : Clé étrangère pour les fichiers météo.
- building_id : Clé étrangère pour train.csv
- primary_use : Indicateur de la catégorie principale d'activités pour le bâtiment basé sur les définitions de type de propriété EnergyStar
- square_feet : Surface de plancher brute du bâtiment
- year_built : Année d'ouverture du bâtiment
- floor_count : Nombre d'étages du bâtiment

In [ ]:
building_metadata.shape

In [ ]:
building_metadata.head()

In [ ]:
building_metadata.describe()

In [ ]:
building_metadata.info()

les variables years_built et floor_count ont beaucoup de valeurs manquantes ( 774 pour years_built  et 1094 pour floor_count)

#### Variable primary_use

In [ ]:
building_metadata.primary_use.unique()

16 types de catégorie principale d'activités pour le bâtiment, la variable primary_use peut être pris comme une varibles catégorielle.


In [ ]:
sns.countplot(y="primary_use",data=building_metadata ,color="salmon")

On remarque que la majorité des acitivités des bâtiments est lié à l'éducation, et la minorité est lié a des lieux de culte religieux.  

#### Variable site_id

In [ ]:
site_build = building_metadata.groupby('site_id').building_id.size()
sns.barplot(x=site_build.index , y= site_build,color="blue")
plt.ylabel("Number of building")
del site_build

On remarque le site 3 a le plus grand nombre de batiments, et le site 11 en a le moins.

#### Variable square_feet, years_built et floor_count

In [ ]:
fig, axes = plt.subplots(3,1,figsize=(10,10)) 
columns = building_metadata.drop(["primary_use","site_id","building_id"],axis=1).columns
for i,col in enumerate(list(columns)):
    plot = building_metadata.boxplot(col, by="site_id", ax=axes.flatten()[i])

plt.tight_layout() 

plt.show()

On remarque que le site 0 est un site avec des bâtiments récents, et le site 4 qu'en a lui a des baitments assez anciens. Par rapport au nombres d'étages, le site 8 ne possède pas beaucoup d'étages contrairement au site 7 qui a des bâtiments avec plus d'étages.

In [ ]:
building_metadata.hist(column ="year_built",bins=40)

Beaucoup de batiments datent des années 1975.

In [ ]:
building_metadata['year_built'].describe()

### C- Analyse des données méteorologiques (weather_data)

In [ ]:
weather_train.shape

La base d'apprentissage des données météorologiques comportent au total 139773 observations évaluées sur 8 variables que sont: la température de l'air, la couverture en nuage, la température de la rosée, le niveau des pluies, la pression du niveau de mer, la direction et la vitesse du vent. Chaque observation (ligne) corresponds au relevé météorologique par heure de chaque site. Ainsi, chaque site a 24 lignes pour les 8 données méteo recueillies.

In [ ]:
weather_train.info()

Toutes les données de cette base n'ont pas pu être recueillies. Il y a des valeurs manquantes pour toutes les variables mesurées sauf évidemment les heures de recueil. Les heures de recueil **timestamp** seront transformées en format date.

In [ ]:
weather_train.describe()

En analysant les statistiques descriptives des variables de la table weather on peut soupçonner la présence de variables abérrantes. En effet, lorsque nous prenons les valeurs de températures, nous avons des valeurs extrêmes (min et max) très éloignées avec de fortes dispersions (std=10,62 et 9,79).
La variable precip_depth_1_hr contient à priori des variables abbérantes car la valeur minimum est de -1 alors que la valeur maximale est de 343 avec une valeur moyenne de 0.98 et des quartiles nulles.
De même, les variables de mesure du vent (wind_direction et wind_speed) présentent des caractéristiques similaires aux précédentes.
Nous pouvons vérifier la présence de valeurs abbérantes grâce au box-plot ci-dessous.

In [ ]:
fig, axes = plt.subplots(7,1,figsize=(10,30)) 
columns = weather_train.drop(["site_id","timestamp"],axis=1).columns
for i,col in enumerate(list(columns)):

    plot = weather_train.boxplot(col, by="site_id", ax=axes.flatten()[i])

plt.tight_layout() 

plt.show()

La représentation des box-plot confirme bien la présence de valeurs abbérantes pour toutes les variables d'étude de la table sauf la variable wind_direction.
Aussi, en général les données météorologiques varient en fonction des sites étudiés.
Par exemple, les sites 0, 8 et 9 ont en moyenne des températures (ambiante et de la rosée) élevées contrairement aux sites 7 et 11 avec des températures (ambiante et de la rosée) basses.
Par ailleurs, on peut également noter que nous avons très peu de données pour les variables precip_depth_1_hr et cloud_coverage. Sur le site 11, nous n'avons pas de données pour la variable cloud_coverage et sur les sites 1, 5 et 12 nous n'avons pas de données pour la variable precip_depth_1_hr.

Etudions la corrélation entre les différentes variables de la base weather_train

In [ ]:
sns.heatmap(weather_train.corr(),linewidths=.5,annot=True)

On obtient ci-dessus une matrice des valeurs de corrélation représentée par des couleurs. On remarque essentiellement qu'il y a une forte corrélation entre les variables **dew_temperature** et **air_température**